# *CROSS VAL*

In [1]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd

In [2]:
import pickle
with open('credit.pkl', 'rb') as f:
    X_credit_treinamento, Y_credit_treinamento, X_credit_teste, Y_credit_teste = pickle.load(f)

In [3]:
X_credit = np.concatenate((X_credit_treinamento, X_credit_teste), axis=0)
X_credit.shape

(2000, 3)

In [4]:
Y_credit = np.concatenate((Y_credit_treinamento, Y_credit_teste), axis=0)
Y_credit.shape

(2000,)

In [5]:
resultados_arvore = []
resultados_random_forest = []
resultados_knn = []
resultados_logistica = []
resultados_svm = []
resultados_neural = []

for i in range(30):
    kfold = KFold(n_splits=10, shuffle=True, random_state=i)
    
    ## ARVORE DE DECISAO ##
    arvore = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=1, min_samples_split=5, splitter='best')
    scores = cross_val_score(arvore, X_credit, Y_credit, cv=kfold)
    resultados_arvore.append(scores.mean())
    
    ## FLORESTA RANDOMICA ##
    random_forest = RandomForestClassifier(criterion='entropy', min_samples_leaf=1, min_samples_split=5, n_estimators=10)
    scores = cross_val_score(random_forest, X_credit, Y_credit, cv=kfold)
    resultados_random_forest.append(scores.mean())
    
    ## VIZINHO MAIS PRÓXIMO ##
    knn = KNeighborsClassifier()
    scores = cross_val_score(knn, X_credit, Y_credit, cv=kfold)
    resultados_knn.append(scores.mean())
    
    ## REGRESSÃO LOGÍSTICA ##
    logistica = LogisticRegression(C=1.0, solver='lbfgs', tol=0.0001)
    scores = cross_val_score(logistica, X_credit, Y_credit, cv=kfold)
    resultados_logistica.append(scores.mean())
    
    ## MAQUINAS DE VETORES DE SUPORTE ##
    svm = SVC(kernel='rbf', C=2.0)
    scores = cross_val_score(svm, X_credit, Y_credit, cv=kfold)
    resultados_svm.append(scores.mean())
    
    ## REDES NEURAIS ##
    rede_neural = MLPClassifier(activation='relu', batch_size=56, solver='adam')
    scores = cross_val_score(rede_neural, X_credit, Y_credit, cv=kfold)
    resultados_neural.append(scores.mean())
    
    

c:\Users\bruno\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\bruno\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\bruno\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\bruno\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

In [6]:
resultados = pd.DataFrame({
                            'Arvore': resultados_arvore, 
                            'Random Forest': resultados_random_forest, 
                            'KNN': resultados_knn, 
                            'Logistica': resultados_logistica, 
                            'SVM': resultados_svm, 
                            'ANN': resultados_neural
                            })

In [7]:
resultados.describe()

,Arvore,Random Forest,KNN,Logistica,SVM,ANN
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,0.987267,0.983850,0.980050,0.947000,0.985083,0.997100
std,0.001721,0.002320,0.001533,0.000743,0.001280,0.000423
min,0.983000,0.980000,0.977000,0.945500,0.982000,0.996500
25%,0.986125,0.982500,0.979000,0.946500,0.984125,0.997000
50%,0.987500,0.984500,0.980000,0.947000,0.985000,0.997000
75%,0.988375,0.985875,0.981000,0.947500,0.986375,0.997500
max,0.990500,0.987500,0.982500,0.948500,0.987500,0.998000


In [8]:
resultados.var()

Arvore           2.960920e-06
Random Forest    5.381897e-06
KNN              2.350862e-06
Logistica        5.517241e-07
SVM              1.639368e-06
ANN              1.793103e-07
dtype: float64

In [9]:
(resultados.std() / resultados.mean()) * 100

Arvore           0.174293
Random Forest    0.235797
KNN              0.156446
Logistica        0.078435
SVM              0.129977
ANN              0.042468
dtype: float64

In [10]:
alpha = 0.05

In [11]:
from scipy.stats import shapiro

In [12]:
shapiro(resultados_arvore), shapiro(resultados_random_forest), shapiro(resultados_knn), shapiro(resultados_logistica), shapiro(resultados_svm), shapiro(resultados_neural)

(ShapiroResult(statistic=np.float64(0.9713487378546231), pvalue=np.float64(0.57670461625493)),
 ShapiroResult(statistic=np.float64(0.9320767538240715), pvalue=np.float64(0.055766442961124645)),
 ShapiroResult(statistic=np.float64(0.9584024823930501), pvalue=np.float64(0.2818135378764127)),
 ShapiroResult(statistic=np.float64(0.9309147727892649), pvalue=np.float64(0.051930740130906254)),
 ShapiroResult(statistic=np.float64(0.9622985607506823), pvalue=np.float64(0.3541579284945434)),
 ShapiroResult(statistic=np.float64(0.8677082204800612), pvalue=np.float64(0.001490238647395843)))

In [13]:
from scipy.stats import f_oneway

In [14]:
p = f_oneway(resultados_arvore, resultados_random_forest, resultados_knn, resultados_logistica, resultados_svm, resultados_neural)

In [15]:
p

F_onewayResult(statistic=np.float64(4064.1922838353294), pvalue=np.float64(4.017233492501983e-178))